In [91]:
import pandas as pd

from surprise import Dataset, Reader, KNNBasic, dump
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import accuracy

In [73]:
books = pd.read_csv('Books.csv')
ratings = pd.read_csv('Ratings.csv')
users = pd.read_csv('Users.csv')

df = users.merge(ratings, on='User-ID')
df = df.merge(books, on='ISBN')

df = df[df['Book-Rating'] != 0]

C:\Users\admin\AppData\Local\Temp\ipykernel_2356\3431223117.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv')


In [74]:
user_rating_count = df.groupby('User-ID')['Book-Rating'].count()
users_more_than_20_ratings = user_rating_count[user_rating_count > 10].index
df = df[df['User-ID'].isin(users_more_than_20_ratings)]

book_rating_count = df.groupby('ISBN')['Book-Rating'].count()
books_more_than_20_ratings = book_rating_count[book_rating_count > 10].index
df = df[df['ISBN'].isin(books_more_than_20_ratings)]

In [75]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df[['User-ID', 'ISBN', 'Book-Rating']], reader=reader)

param_grid = {
    'bsl_options': {
        'method': ['als', 'sgd'],
        'reg': [1, 2],
    },
    'k': [2, 3],
    'sim_options': {
        'name': ['msd', 'cosine'],
        'min_support': [1, 5],
        'user_based': [False],
    },
}

gs = GridSearchCV(KNNBasic, param_grid=param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matr

In [76]:
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

1.7587618010652968
{'bsl_options': {'method': 'als', 'reg': 1}, 'k': 3, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}


In [77]:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [87]:
book_id = '0446607657'
item_inner_id = algo.trainset.to_inner_iid(book_id)
neighbors = algo.get_neighbors(item_inner_id, k=5)

neighbor_books = [algo.trainset.to_raw_iid(inner_id) for inner_id in neighbors]

In [88]:
neighbor_books

['0316693251', '0446612545', '0385504209', '0440214041', '0142001740']

In [89]:
df[df['ISBN'] == '0316693251']['Book-Title'].unique()

array(['Roses Are Red'], dtype=object)

In [92]:
model_filename = 'collab_filtering_model.pkl'
dump.dump(model_filename, algo)

In [93]:
df.to_csv('book_rating.csv', index=False)

In [96]:
str(df[df['ISBN'] == '0316693251']['Book-Title'].unique())

"['Roses Are Red']"